In [ ]:
import os 
from dotenv import load_dotenv
from azure.core.credentials import AzureKeyCredential
from agent_framework import ChatAgent
from agent_framework.azure import AzureAIAgentClient
from azure.ai.projects.aio import AIProjectClient
from azure.identity.aio import DefaultAzureCredential,AzureCliCredential
from agent_framework import Workflow, WorkflowBuilder, WorkflowContext, WorkflowOutputEvent, executor, handler, WorkflowViz
from typing import List,Any 
from agent_framework import ChatResponse
from agent_framework import WorkflowBuilder ,WorkflowViz

In [ ]:
load_dotenv()
project_endpoint = os.getenv("AI_FOUNDRY_PROJECT_ENDPOINT")
model = os.getenv("AI_FOUNDRY_DEPLOYMENT_NAME")


In [ ]:
async def create_agent(agent_name: str, agent_instructions: str) -> ChatAgent:
    credential = DefaultAzureCredential()
    ai_project_client = AIProjectClient(endpoint=project_endpoint, credential=credential)
    azure_ai_agent_client = AzureAIAgentClient(
        ai_project_client=ai_project_client,
        project_endpoint=project_endpoint,
        deployment_name=model,
        credential=credential
    )
    agent = ChatAgent(
        agent_name=agent_name,
        agent_instructions=agent_instructions,
        azure_ai_agent_client=azure_ai_agent_client
    )
    return agent

In [ ]:
researcher_agent = await create_agent(
    agent_name= "Researcher agent",
    agent_instructions= "You are an expert research agent. Your task is to gather relevant information based on user queries."
)

In [ ]:
writer_agent = await create_agent(  
    agent_name= "Writer agent",
    agent_instructions= "You are an expert writer agent. Your task is to create well-structured and engaging content based on the research provided."
)

In [ ]:
@executor(id = "run_researcher_agent")
async def run_researcher_agent(query: str,
                               ctx: WorkflowContext[str]) -> None:
    response = await researcher_agent.run(query)

    await ctx.send_message(str(response))

In [ ]:

workflow = (
    WorkflowBuilder()
    .add_edge(run_researcher_agent, run_writer_agent)
    .set_start_executor(run_researcher_agent)
    .build()
)

In [ ]:
viz = WorkflowViz(workflow)

mermaid_content = viz.to_mermaid()

In [ ]:
async def main():
    async for event in workflow.run_stream("write an essay about the impact of AI on society"):
        print(f"Event: {event}")
        if isinstance(event, WorkflowOutputEvent):
            print(f"Workflow completed with result: {event.data}")

await main()